# Procesamiento de lenguaje natural
## Desafío 2


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Directorio de los dataset de libros
os.listdir("./libros_dataset/")

['Fundacion.txt']

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('libros_dataset/Fundacion.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_21115/1480457263.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('libros_dataset/Fundacion.txt', sep='/n', header=None)


,0
0,TRILOGÍA DE LA FUNDACION
1,FUNDACIÓN
2,FUNDACIÓN E IMPERIO
3,SEGUNDA FUNDACIÓN
4,Reunión en un solo volumen de las tres novelas...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 26966


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2024-05-08 19:10:09.668943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 19:10:09.760220: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 19:10:10.153063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jorge/miniconda3/envs/tensorflow/lib/:/home/jorge/miniconda3/envs/tensorflow/lib/python3.9/site-packages/tensorrt/
2024-05-08 19:10:10.153131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['trilogía', 'de', 'la', 'fundacion'], ['fundación']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=12,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 26966


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4351


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 154946.59375
Loss after epoch 1: 133281.59375
Loss after epoch 2: 130434.1875
Loss after epoch 3: 126205.375
Loss after epoch 4: 124449.6875
Loss after epoch 5: 122004.5
Loss after epoch 6: 118234.8125
Loss after epoch 7: 117018.0
Loss after epoch 8: 107289.25
Loss after epoch 9: 103792.125
Loss after epoch 10: 105683.5
Loss after epoch 11: 103053.75
Loss after epoch 12: 101733.75
Loss after epoch 13: 99318.375
Loss after epoch 14: 99180.75
Loss after epoch 15: 99254.5
Loss after epoch 16: 97193.875
Loss after epoch 17: 95754.0
Loss after epoch 18: 94694.625
Loss after epoch 19: 86641.25
Loss after epoch 20: 89180.0
Loss after epoch 21: 87788.25
Loss after epoch 22: 86088.25
Loss after epoch 23: 85850.0
Loss after epoch 24: 84232.75
Loss after epoch 25: 84884.25
Loss after epoch 26: 85684.25
Loss after epoch 27: 81805.75
Loss after epoch 28: 83967.25
Loss after epoch 29: 82673.5
Loss after epoch 30: 80931.0
Loss after epoch 31: 80825.25
Loss after epoch 32: 80187.0


(6656422, 10737500)

### 4 - Ensayar

Ensayare el modelo usando las siguientes palabras:

- Imperio.
- Seldon.
- Gaal
- Fundación.
- Psicohistoria.
- Enciclopedia.
- Mulo.
- Términus.
- Imperial.
- Trántor

### Similitudes

Mostraré las 5 palabras mas similes.

In [13]:
# Palabras que MÁS se relacionan con "Imperio":
w2v_model.wv.most_similar(positive=["imperio"], topn=5)


[('humanidad', 0.4227783679962158),
 ('excesivamente', 0.4138237535953522),
 ('existido', 0.3911753296852112),
 ('comenzado', 0.3872603178024292),
 ('galáctico', 0.3863332271575928)]

In [14]:
# Palabras que MÁS se relacionan con "Seldon":
w2v_model.wv.most_similar(positive=["seldon"], topn=5)

[('hari', 0.5614129900932312),
 ('8', 0.47081875801086426),
 ('dornick', 0.44619622826576233),
 ('previó', 0.4159797132015228),
 ('inminente', 0.4141344428062439)]

In [15]:
# Palabras que MÁS se relacionan con "Gaal":
w2v_model.wv.most_similar(positive=["gaal"], topn=5)

[('dornick', 0.42411357164382935),
 ('escrita', 0.3889870047569275),
 ('8', 0.38569343090057373),
 ('aguardó', 0.37411415576934814),
 ('—riose', 0.3728770613670349)]

In [16]:
# Palabras que MÁS se relacionan con "Fundación":
w2v_model.wv.most_similar(positive=["fundación"], topn=5)

[('segunda', 0.47963017225265503),
 ('isaac', 0.41617897152900696),
 ('actualmente', 0.38129740953445435),
 ('convencido', 0.36477819085121155),
 ('paradero', 0.3636091947555542)]

In [17]:
# Palabras que MÁS se relacionan con "Psicohistoria":
w2v_model.wv.most_similar(positive=["psicohistoria"], topn=5)

[('distintas', 0.47944939136505127),
 ('previó', 0.4661312997341156),
 ('predice', 0.459117591381073),
 ('reacciones', 0.44807684421539307),
 ('sociales', 0.4464583992958069)]

In [18]:
# Palabras que MÁS se relacionan con "Enciclopedia":
w2v_model.wv.most_similar(positive=["enciclopedia"], topn=5)

[('proceden', 0.491924524307251),
 ('espiral', 0.47742587327957153),
 ('edición', 0.4708521068096161),
 ('7', 0.426017701625824),
 ('hagdin', 0.415183961391449)]

In [19]:
# Palabras que MÁS se relacionan con "Mulo":
w2v_model.wv.most_similar(positive=["mulo"], topn=5)

[('definitiva', 0.3691677153110504),
 ('mutación', 0.33885031938552856),
 ('potencial', 0.3327149450778961),
 ('veloces', 0.3323534429073334),
 ('maligno', 0.32774582505226135)]

In [20]:
# Palabras que MÁS se relacionan con "Términus":
w2v_model.wv.most_similar(positive=["términus"], topn=5)

[('establecida', 0.37652823328971863),
 ('indefensos', 0.36423400044441223),
 ('áreas', 0.3573697805404663),
 ('periódicos', 0.3554331958293915),
 ('llegará', 0.35110142827033997)]

In [21]:
# Palabras que MÁS se relacionan con "Imperial":
w2v_model.wv.most_similar(positive=["imperial"], topn=5)

[('terrenos', 0.4944378733634949),
 ('biblioteca', 0.4473087191581726),
 ('majestad', 0.41380083560943604),
 ('etiqueta', 0.399430513381958),
 ('excesivamente', 0.3835916817188263)]

In [22]:
# Palabras que MÁS se relacionan con "Trántor":
w2v_model.wv.most_similar(positive=["trántor"], topn=5)

[('santanni', 0.40551358461380005),
 ('setenta', 0.3521673381328583),
 ('humanidad', 0.34293192625045776),
 ('enemiga', 0.3413773775100708),
 ('agrícola', 0.34000030159950256)]

### Diferencias

Mostraré las 5 palabras mas diferentes.

In [23]:
# Palabras que MENOS se relacionan con "Imperio":
w2v_model.wv.most_similar(negative=["imperio"], topn=5)

[('observó', 0.08237320184707642),
 ('tratando', 0.024252666160464287),
 ('glacial', 0.017111940309405327),
 ('acerca', 0.016870904713869095),
 ('leer', 0.0156650822609663)]

In [24]:
# Palabras que MENOS se relacionan con "Seldon":
w2v_model.wv.most_similar(negative=["seldon"], topn=5)

[('vestido', 0.04233042150735855),
 ('dando', 0.04131508246064186),
 ('tus', 0.023455817252397537),
 ('memoria', 0.01902143284678459),
 ('extraordinario', 0.015026352368295193)]

In [25]:
# Palabras que MENOS se relacionan con "Gaal":
w2v_model.wv.most_similar(negative=["gaal"], topn=5)

[('flota', 0.04958363249897957),
 ('cuerpo', 0.048968587070703506),
 ('además', 0.048146408051252365),
 ('haga', 0.03858568146824837),
 ('pobre', 0.028668737038969994)]

In [26]:
# Palabras que MENOS se relacionan con "Fundación":
w2v_model.wv.most_similar(negative=["fundación"], topn=5)

[('delgado', 0.03525850549340248),
 ('frío', 0.03269113972783089),
 ('receptor', 0.031765516847372055),
 ('saltar', 0.02349809743463993),
 ('apareció', 0.01957077719271183)]

In [27]:
# Palabras que MENOS se relacionan con "Psicohistoria":
w2v_model.wv.most_similar(negative=["psicohistoria"], topn=5)

[('muchacho', 0.03969757258892059),
 ('tus', 0.032281264662742615),
 ('fugitivo', 0.0319315604865551),
 ('habitación', 0.024393867701292038),
 ('llevar', 0.013972575776278973)]

In [28]:
# Palabras que MENOS se relacionan con "Enciclopedia":
w2v_model.wv.most_similar(negative=["enciclopedia"], topn=5)

[('cierta', 0.04042782634496689),
 ('salto', 0.03727056831121445),
 ('murmullo', 0.03212190419435501),
 ('repentina', 0.027919702231884003),
 ('bayta', 0.025989489629864693)]

In [29]:
# Palabras que MENOS se relacionan con "Mulo":
w2v_model.wv.most_similar(negative=["mulo"], topn=5)

[('lista', 0.03869973495602608),
 ('ausencia', 0.033957649022340775),
 ('particular', 0.029606662690639496),
 ('ciudad', 0.019026415422558784),
 ('placer', 0.01889760047197342)]

In [30]:
# Palabras que MENOS se relacionan con "Términus":
w2v_model.wv.most_similar(negative=["términus"], topn=5)

[('general', 0.04690992087125778),
 ('acaso', 0.03627822920680046),
 ('sangre', 0.03137945383787155),
 ('falta', 0.031003184616565704),
 ('bajó', 0.03025730326771736)]

In [31]:
# Palabras que MENOS se relacionan con "Imperial":
w2v_model.wv.most_similar(negative=["imperial"], topn=5)

[('reflexionó', 0.05053584650158882),
 ('elemento', 0.04028680920600891),
 ('improviso', 0.028240686282515526),
 ('tono', 0.023824721574783325),
 ('oído', 0.021191732957959175)]

In [32]:
# Palabras que MENOS se relacionan con "Trántor":
w2v_model.wv.most_similar(negative=["trántor"], topn=5)

[('torpe', 0.03754197061061859),
 ('leer', 0.037362970411777496),
 ('forma', 0.03433249890804291),
 ('ansiedad', 0.031703293323516846),
 ('captó', 0.02561851777136326)]

### 5 - Visualizar agrupación de vectores

In [33]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [34]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [35]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show() # esto para plotly en colab

In [36]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)